In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import json
import src.scraping as sr
import numpy as np

### Getting the data from "Jornada Virtual ACB"

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://jv.acb.com/es/101266/cartadetiro')
driver.implicitly_wait(10)

In [5]:
final = sr.season(1, 34, driver)
data_items = final.items()
data_list = list(data_items)

df = pd.DataFrame(data_list)

In [6]:
df.head()

,0,1
0,1,"[{'A. Della Valle': ['#local-out', '104.418306..."
1,2,"[{'J. Zyskowski': ['#local-in', '24.75', '126...."


In [227]:
df.to_csv("./data/shots_v1.csv")

### Getting general data about every player.

In [3]:
# Choosing url[0], html[0], we will get general info. To get stats info, you have to use:
# url[1], html[1] for averages stats
# url[2], html[1] for per_minute stats
# url[3], html[1] for per 40 stats

url = ['https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/rosters/2021',
       'https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Averages/All/All',
       'https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_Minute/All/All/points/All/desc/1/Regular_Season',
       'https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season'
      ]

html = ["tablesaw.tablesaw-swipe.tablesaw-sortable", "tablesaw.compact.tablesaw-swipe"]

In [4]:
columns_name = sr.players_name(url[0], html[0])

In [6]:
stats_list = sr.table_stats(url[0], html[0])

starting loop on option /international/league/4/Spanish-ACB/team/23/Barca/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/473/BAXI-Manresa/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/129/Casademont-Zaragoza/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/16/Club-Joventut-Badalona/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/212/Coosur-Real-Betis/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/33/Herbalife-Gran-Canaria/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/1040/Hereda-San-Pablo-Burgos/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/1039/Iberostar-Tenerife/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/651/Monbus-Obradoiro/rosters/2021
starting loop on option /international/league/4/Spanish-ACB/team/1028/MoraBanc-Andorra/rosters/2021


In [7]:
general_final = np.array_split(stats_list,364)
general_stats = pd.DataFrame(general_final, columns = columns_name)
general_stats.drop(columns = ["#", "Birth City", "NBA Draft Status", "Nationality"], inplace = True)

### Getting all the stats per minute of every player in this season (2020-21).

In [8]:
#Getting the headers from stats in-game of every player. 

headers_player_stats = sr.players_name(url[2], html[1])

In [ ]:
per_minute_stats = sr.table_stats(url[2], html[1])

In [ ]:
#The result of "no_players" will be the number of players whose we have info about. 

no_players = len(per_minute_stats) / 23
final = np.array_split(per_minute_stats, no_players)
acb_dataset_per_minute = pd.DataFrame(final, columns = headers_player_stats)

### Getting all the stats averaged of every player in this season (2020-21).

In [ ]:
average_stats = sr.table_stats(url[1], html[1])

In [ ]:
#The result of "no_players" will be the number of players whose we have info about. 

no_players = len(average_stats) / 23
average_final = np.array_split(average_stats, no_players)
average_stats = pd.DataFrame(average_final, columns = headers_player_stats)

### Getting all the stats per 40 minute of every player in this season (2020-21).

In [28]:
per40_stats = sr.table_stats(url[3], html[1])

starting loop on option /international/league/4/Spanish-ACB/team/649/Acunsa-GBC/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/23/Barca/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/473/BAXI-Manresa/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/129/Casademont-Zaragoza/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/16/Club-Joventut-Badalona/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/212/Coosur-Real-Betis/stats/2021/Per_40/All/All/points/All/desc/1/Regular_Season
starting loop on option /international/league/4/Spanish-ACB/team/33/Herbalife-Gran-Canaria/stats/2021/Per_40/All/All/points/All/desc/1/Regular_

In [29]:
#The result of "no_players" will be the number of players whose we have info about. 

no_players = len(per40_stats) / 23
per40_final = np.array_split(per40_stats, no_players)
per40_stats = pd.DataFrame(per40_final, columns = headers_player_stats)

### Merging general stats with stats tables

In [ ]:
final_per_minute = acb_dataset_per_minute.merge(general_stats, how  = "inner", left_on = "Player", right_on = "Player")

In [ ]:
final_per_minute.to_csv("./data/stats_per_minute.csv")

In [ ]:
final_average = average_stats.merge(general_stats, how  = "inner", left_on = "Player", right_on = "Player")

In [ ]:
final_average.to_csv("./data/acb_stats.csv")

In [30]:
final_per40 = per40_stats.merge(general_stats, how  = "inner", left_on = "Player", right_on = "Player")
final_per40.to_csv("./data/per40_stats.csv")